In [1]:
import os
import sys
sys.path.append('/home/simon/Research/lib/caffe/python/')
import caffe
caffe.set_device(0)
caffe.set_mode_gpu()
sys.path.append('./classification_framework2/')
import Caffe
import Resize
import Crop
import ImageReader
import Classification
import Dataset

import h5py
import numpy as np
import sklearn.linear_model
import sklearn.svm
import random
import pyprind
import sklearn.svm
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
imagelist = './example/imagelist.txt'

architecture = './models/vgg16_alpha_pooling.prototxt'
# ImageNet model
weights = './models/vgg16_alpha_pooling_imagenet.caffemodel'
# INat model
# weights = './models/vgg16_alpha_pooling_inat.caffemodel'

In [3]:
c = Classification.Classification()
c.add_algorithm(ImageReader.ImageReader())
s = 560
c.add_algorithm(Resize.Resize((s,), mode='resize_smaller_side'))
c.add_algorithm(Crop.Crop((s,s),'center'))
c.add_algorithm(Caffe.Caffe(architecture,
                           weights,
                           outblob = 'bilinear_l2',
                           batchsize = 1))

In [5]:
images = open(imagelist, 'rt').read().splitlines()

In [6]:
features = np.stack(c.compute_all(images))

0%  100%
/home/simon/bin/conda/envs/p3/lib/python3.5/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
[###] | ETA: 00:00:00
Total time elapsed: 00:00:00


In [7]:
features.shape

(3, 8192, 1, 1)

In [13]:
np.savez('./features.npz',features=features)

### Test area

In [4]:
basedir = '/home/simon/Datasets/CUB_200_2011/'
d = Dataset.Dataset()
d.read_from_file('%s/imagelist_absolute.txt'%basedir, 'imagepaths','string')
d.read_from_file('%s/labels.txt'%basedir,'labels','int')
d.read_from_file('%s/tr_ID.txt'%basedir,'split_assignments','int')

In [5]:
train_ids = np.where(d.split_assignments)[0]
test_ids = np.where(np.invert(np.array(d.split_assignments).astype(bool)))[0]

In [6]:
#Calc the features
def calc_features(pipeline, images):
    out_shape = (len(images), ) + pipeline.compute(images[0]).shape
    feats = np.zeros(tuple(out_shape),dtype=np.float32)
    print('feats will occupy %.4f GiB'%(feats.nbytes/1024/1024/1024))
    for idx,impath in enumerate(pyprind.prog_bar(images)):
        feats[idx,...] = c.compute(impath)
    return feats

In [7]:
train_fv = calc_features(c, np.array(d.imagepaths)[train_ids])
test_fv = calc_features(c, np.array(d.imagepaths)[test_ids])

/home/simon/bin/conda/envs/p3/lib/python3.5/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
0%                          100%


feats will occupy 0.1829 GiB


[##############################] | ETA: 00:00:00
Total time elapsed: 00:20:28
0%                          100%


feats will occupy 0.1768 GiB


[##############################] | ETA: 00:00:00
Total time elapsed: 00:19:56


In [8]:
train_fv.shape

(5994, 8192, 1, 1)

In [9]:
task = dict(train_feats=train_fv.reshape((len(train_fv),-1)),
                                test_feats=test_fv.reshape((len(test_fv),-1)),
                                train_Y=np.array(d.labels)[train_ids],
                                test_Y=np.array(d.labels)[test_ids])

In [10]:
# for comparison: explicit feature transform of linear kernel
#model = sklearn.multiclass.OneVsRestClassifier(sklearn.svm.SVC(C=1,kernel='linear'),n_jobs=-1)
model = sklearn.svm.LinearSVC(C=1)
#model = sklearn.linear_model.LogisticRegression(C=10000,solver='lbfgs',multi_class='multinomial')
model.fit(task['train_feats'],task['train_Y'])
model.score(task['test_feats'],task['test_Y'])

0.80272695892302381